In [1]:
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor

In questa esercitazione, vedremo come codificare rapidamente le categorical feature presenti in un insieme di dati, e come sfruttare un metodo di ricerca ottimizzato per identificare i migliori iperparametri da applicare ad uno stimatore.

## Parte 1: caricamento e preprocessing dei dati

Useremo gli stessi identici dati dell'esercitazione precedente.

In [2]:
df = pd.read_csv('../dataset/train.csv')
df = df[['Sex', 'Age', 'Fare']]
df.dropna(axis=0, inplace=True)

Ricordiamo che, nell'esercitazione precedente, abbiamo dovuto utilizzare un oggetto di tipo [`OrdinalEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) per trasformare i dati da stringa in numero.

Questo approccio però risulta essere inefficace quando ci troviamo di fronte ad un dataframe in cui coesistono più categorical feature. In questi casi, Scikit Learn ci mette a disposizione un oggetto chiamato [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) il quale permette di codificare contemporaneamente un insieme di feature.

In particolare, dovremo specificare ciascuna delle feature da trasformare sotto forma di tupla del tipo:

`(name, transformer, columns)`

dove `name` rappresenta il nuovo nome che sarà associato alla feature, `transformer` il tipo di trasformazione che vogliamo effettuare, e `columns` un indice che ci permette di identificare la colonna da modificare.

In [10]:
ct = ColumnTransformer(
    [('Sex_', OrdinalEncoder(), ['Sex'])],
    remainder='passthrough')
data = ct.fit_transform(df)
X = data[:, :2]
y = data[:, 2]

In [11]:
X

array([[ 1., 22.],
       [ 0., 38.],
       [ 0., 26.],
       ...,
       [ 0., 19.],
       [ 1., 26.],
       [ 1., 32.]])

## Parte 2: ottimizzazione degli iperparametri

Finora abbiamo usato un approccio *trial & error* alla ricerca della combinazione ottimale di iperparametri da usare per uno stimatore.

Ovviamente, questo approccio risulta essere subottimale, ed è stata sviluppata un'apposita branca del machine learning, chiamata *hyperparameters tuning*, dedicata proprio a "semplificarci la vita".

Esistono molte tecniche di hyperparameters tuning; alcune tra di esse sono ovviamente disponibili in Scikit Learn. In particolare, proveremo ad usare la *grid search*, ovvero una ricerca "a griglia", che prevede che siano provate tutte le possibili combinazioni di iperparametri prima della scelta dei migliori. Ad ogni prova, corrisponderà un determinato scoring (ad esempio, in termini di accuracy o di MSE); al termine delle prove, sarà selezionata la combinazione che ha ottenuto il risultato migliore.

Per implementare la grid search, Scikit Learn ci offre la classe [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) che, come suggerisce il nome stesso, implementa in automatico la cross validazione. Proviamo ad usarla su un albero decisionale, scegliendo come parametri possibili una serie di [criteri di valutazione per lo split](https://www.analyticsvidhya.com/blog/2020/06/4-ways-split-decision-tree/) (ovvero, come i nodi vengono suddivisi fino a che non sono considerati *omogenei*) e la massima profondità raggiungibile dall'albero di regressione.

In [12]:
dt = DecisionTreeRegressor()
pars = {
    'criterion': ['mse', 'friedman_mse', 'mae', 'poisson'],
    'max_depth': list(range(1, 11))
}
rgr = GridSearchCV(dt, pars, cv=10)
rgr.fit(X, y)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae', 'poisson'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

Una volta terminato l'addestramento, che avviene sempre e comunque usando il metodo `fit()`, è possibile consultare diversi attributi, come ad esempio quello relativo ai migliori parametri individuati ed al miglior punteggio ottenuto.

In [14]:
print('Migliori parametri selezionati:\t{}'.format(rgr.best_params_))
print('Miglior punteggio di MSE:\t{}'.format(rgr.best_score_))

Migliori parametri selezionati:	{'criterion': 'friedman_mse', 'max_depth': 2}
Miglior punteggio di MSE:	0.0018751601472768442


Interessante infine notare come l'interfaccia offerta dagli oggetti di classe `GridSearchCV` sia coerente con quella dei predittori. Potremo quindi chiamare i principali metodi che ci aspettiamo di invocare su un classificatore o un regressore, come ad esempio `predict()`.

In [17]:
print('Prezzo del biglietto pagato da un uomo di venti anni: {}'.format(
    round(rgr.predict([[0, 12]])[0], 2))) 

Prezzo del biglietto pagato da un uomo di venti anni: 38.69


## Esercizio

Proviamo ad usare la `GridSearchCV` su un random forest. Individuiamo come parametri il numero di alberi nella foresta, il criterio e la massima profondità di ciascun albero.

In [18]:
rf = RandomForestRegressor()
pars = {
    'n_estimators': [50, 100, 150, 200],
    'criterion': ['mae', 'mse'],
    'max_depth': list(range(1, 10))
}
clf = GridSearchCV(rf, pars)
clf.fit(X, y)
print('Migliori parametri selezionati:\t{}'.format(clf.best_params_))
print('Miglior punteggio di MSE:\t{}'.format(clf.best_score_))

Migliori parametri selezionati:	{'criterion': 'mse', 'max_depth': 2, 'n_estimators': 150}
Miglior punteggio di MSE:	0.011150434020413847
